In [14]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import interpolate

In [15]:
#Stuff from class in this cell

sig=0.1

def fun(x):
    return 1.0/(1.0+x**2)

def fun2(x):
    return 1.0+np.exp(-0.5*x**2/(sig**2))

def simple_integrate_class(fun,a,b,tol): #from class
    x=np.linspace(a,b,5)
    dx=(b-a)/4.0
    #np.median(np.diff(x))
    y=fun(x)
    neval=len(x) #let's keep track of function evaluations
    f1=(y[0]+4*y[2]+y[4])/6.0*(b-a)
    f2=(y[0]+4*y[1]+2*y[2]+4*y[3]+y[4])/12.0*(b-a)
    myerr=np.abs(f2-f1)
    print([a,b,f1,f2])
    if (myerr<tol):
        #return (f2)/1.0,myerr,neval
        return (16.0*f2-f1)/15.0,myerr,neval
    else:
        mid=0.5*(b+a)
        f_left,err_left,neval_left=simple_integrate_class(fun,a,mid,tol/2.0)
        f_right,err_right,neval_right=simple_integrate_class(fun,mid,b,tol/2.0)
        neval=neval+neval_left+neval_right
        f=f_left+f_right
        err=err_left+err_right
        return f,err,neval

In [64]:
#Results from class

f,err,neval=simple_integrate_class(np.exp,-1,1,1e-3);

pred=np.exp(1)-np.exp(-1)

#f,err,neval=simple_integrate_class(fun,-1,1,1e-4);pred=np.arctan(1)-np.arctan(-1)
#a=-5;b=5;f,err,neval=simple_integrate_class(fun2,a,b,1e-4);pred=(b-a)+np.sqrt(2*np.pi)*sig
print('f,err,neval are ' + repr([f,err,neval])+' with err ' + repr(np.abs(f-pred)))

[-1, 1, 2.362053756543496, 2.3511948318802554]
[-1, 0.0, 0.6323336800036626, 0.6321341753205323]
[0.0, 1, 1.7188611518765928, 1.7183188419217472]
[0.0, 0.5, 0.6487352447875911, 0.648722148950655]
[0.5, 1, 1.069583597134156, 1.0695620057492419]
f,err,neval are [2.350402717226764, 0.00023419190498064069, 25] with err 3.299391613253988e-07


In [66]:
#New class here for "smarter" integration with less function calls
class Integrator:
    
    def __init__(self):
        self.y_left = 0.0
        self.y_right = 0.0
        self.y_mid = 0.0
        self.mid = 0.0
        self.y = np.zeros(5)
        self.neval = 0
    
    def self.less_simple_integrate(self, fun,a,b,tol, lr="none"): #Doesn't call fun(x) multiple times for same x
        x=np.linspace(a,b,5)
        dx=(b-a)/4.0
        
        if lr == "l":
            self.y[0] = self.y_left #Left bound is the same
            self.y[4] = self.y_mid #Old mid is now right bound
            self.y[1] = fun(x[1])
            self.y[2] = fun(x[2])
            self.y[3] = fun(x[3])
            self.neval +=3
        elif lr == "r":
            self.y[0] = self.mid #Old mid is the new left bound
            self.y[4] = self.y_right #Right bound is the same 
            self.y[1] = fun(x[1])
            self.y[2] = fun(x[2])
            self.y[3] = fun(x[3])
            self.neval+=3
        else:
            self.y=fun(x)
            self.neval+=5
            y_left = self.y[0]
            y_right = self.y[4]
            y_mid = self.y[2]
            

        #Shouldn't have to evaluate left, mid, right again when calling less_simple_integrate
        self.y[1]=fun(x[1])
        self.y[3]=fun(x[3])

        f1=(self.y[0]+4*self.y[2]+self.y[4])/6.0*(b-a)
        f2=(self.y[0]+4*self.y[1]+2*self.y[2]+4*self.y[3]+self.y[4])/12.0*(b-a)

        myerr=np.abs(f2-f1)

        print([a,b,f1,f2])
        if (myerr<tol):
            #return (f2)/1.0,myerr,neval
            return (16.0*f2-f1)/15.0,myerr,neval
        else:
            self.mid=0.5*(b+a)

            f_left,err_left,neval_left=self.less_simple_integrate(fun,a,self.mid,tol/2.0, "l")
            f_right,err_right,neval_right=self.less_simple_integrate(fun,self.mid,b,tol/2.0, "r")

            self.neval=self.neval+neval_left+neval_right

            f=f_left+f_right
            err=err_left+err_right

            return f,err,neval


SyntaxError: invalid syntax (<ipython-input-66-bb848c30ef22>, line 12)

In [67]:
smrt = Integrator()
f,err,neval = smrt.less_simple_integrate(np.exp,-1,1,1e-3)

pred=np.exp(1)-np.exp(-1)

#f,err,neval=simple_integrate_class(fun,-1,1,1e-4);pred=np.arctan(1)-np.arctan(-1)
#a=-5;b=5;f,err,neval=simple_integrate_class(fun2,a,b,1e-4);pred=(b-a)+np.sqrt(2*np.pi)*sig
print('f,err,neval are ' + repr([f,err,neval])+' with err ' + repr(np.abs(f-pred)))


[-1, 1, 2.362053756543496, 2.3511948318802554]
[-1, 0.0, 0.40435377314175563, 0.5181442218895788]
[-1, -0.5, 0.15745551758033824, 0.19805112076133433]
[-1, -0.75, 0.0694770032797514, 0.08698199554976109]
[-1, -0.875, 0.03263380222306658, 0.04080818854936796]
[-1, -0.9375, 0.015814882840962066, 0.019770534118170133]
[-1, -0.96875, 0.007784847904292562, 0.009731297456278184]
[-1, -0.984375, 0.003862132868633372, 0.004827695551564569]
[-1, -0.9921875, 0.0019235379197606276, 0.0024044260685593725]
[-1, -0.99609375, 0.0009598923381155517, 0.0011998658803568107]
[-1, -0.998046875, 0.00047947770040874345, 0.0005993471826691264]
[-1, -0.9990234375, 0.0002396218187933674, 0.00029952728063299596]
[-1, -0.99951171875, 0.00011978166225673456, 0.0001497270787133611]
[-1, -0.999755859375, 5.988352068209886e-05, 7.485440096416532e-05]
[-1, -0.9998779296875, 2.9939932896798354e-05, 3.742491613493981e-05]
[-1, -0.99993896484375, 1.4969509608249422e-05, 1.871188701205446e-05]
[-1, -0.999969482421875, 7.

[-1.0, -1.0, 0.0, 0.0]
[-1.0, -0.9999999999999999, 8.724831314565461e-18, 2.4783826951021284e-17]
[-1.0, -1.0, 0.0, 0.0]
[-1.0, -0.9999999999999999, 8.724831314565461e-18, 2.4783826951021284e-17]
[-1.0, -1.0, 0.0, 0.0]
[-1.0, -0.9999999999999999, 8.724831314565461e-18, 2.4783826951021284e-17]
[-1.0, -1.0, 0.0, 0.0]
[-1.0, -0.9999999999999999, 8.724831314565461e-18, 2.4783826951021284e-17]
[-1.0, -1.0, 0.0, 0.0]
[-1.0, -0.9999999999999999, 8.724831314565461e-18, 2.4783826951021284e-17]
[-1.0, -1.0, 0.0, 0.0]
[-1.0, -0.9999999999999999, 8.724831314565461e-18, 2.4783826951021284e-17]
[-1.0, -1.0, 0.0, 0.0]
[-1.0, -0.9999999999999999, 8.724831314565461e-18, 2.4783826951021284e-17]
[-1.0, -1.0, 0.0, 0.0]
[-1.0, -0.9999999999999999, 8.724831314565461e-18, 2.4783826951021284e-17]
[-1.0, -1.0, 0.0, 0.0]
[-1.0, -0.9999999999999999, 8.724831314565461e-18, 2.4783826951021284e-17]
[-1.0, -1.0, 0.0, 0.0]
[-1.0, -0.9999999999999999, 8.724831314565461e-18, 2.4783826951021284e-17]
[-1.0, -1.0, 0.0, 0.

RuntimeError: maximum recursion depth exceeded